In [2]:
import sys
sys.path.append('../')

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
from tqdm.notebook import tqdm

from src.data import (
    TrainDataset,
    train_collate_fn,
    InferenceDataset,
    inference_collate_fn,
    get_item_list,
    get_item_labels,
    get_pair_list,
    read_words_from_file,
    new_insert_carot
)
from src.models import ( 
    single_model,
    train_model_early_stopping,
    BERT52,
    AttnLSTM52,
    AttnBiLSTM52, 
    LSTMBERT52,
)
from settings import BATCH_SIZE

In [3]:
train_file_path = "../data/raw/train_stresses_labels.txt"
public_file_path = "../data/raw/public_test_stresses.txt"

In [4]:
with open(train_file_path, "r", encoding='utf-8') as file:
    words = file.read().splitlines()

In [10]:
train_words, val_words = [words[i] for i in range(len(words))], [words[i] for i in range(len(words)) if i % 20 == 0]

In [11]:
len(train_words), len(val_words)

(588490, 29425)

In [12]:
with open(public_file_path, "r", encoding='utf-8') as file:
    test_words = file.read().splitlines()

In [16]:
len(test_words) / 588490

0.5000135941137488

In [24]:
%%time
list(set(test_words) & set(list(map(lambda x: x.replace('^', ''), train_words))))

CPU times: total: 172 ms
Wall time: 290 ms


[]

In [26]:
for word in train_words:
    if 'ё' in word:
        print(word)

актёрко^мик
боксёрпрофессиона^л
вентилё^к
вертолётоно^сный
вертолёторемо^нтный
вертолётострое^ние
вертолётострои^тель
вертолётострои^тельный
весё^ловец
взлётнопоса^дочная
вишнё^вец
восьмидесятитрёхле^тие
восьмидесятитрёхле^тия
восьмидесятичетырёхле^тие
восьмидесятичетырёхле^тия
вперёдсмотря^щий
всёравно^
вёлери^т
гололёдообразова^ние
гёрлска^ут
гёрлска^утский
гёрлфрэ^нд
двадцатитрёхле^тие
двадцатичетырёхкра^тный
двадцатичетырёхле^тие
двадцатичетырёхле^тний
двадцатичетырёхуго^льниках
жёлчевыводя^щий
застёжкамо^лния
картё^жничать
киёмо^то
колёснотока^рный
колёсообра^зный
колёсопрока^тный
ла^герлёф
ликёрово^дочный
лёссови^дный
лётчикиспыта^тель
лётчикнаблюда^тель
ма^льмё
мёдолече^ние
мёнхенгла^дбахский
невсётакоднозна^чник
нелёжкоспосо^бный
неплатёжеспосо^бностей
неплатёжеспосо^бности
неплатёжеспосо^бность
неплатёжеспосо^бностью
неплатёжеспосо^бностями
неплатёжеспосо^бностях
перекрёстноопыли^тель
перекрёстноопыля^ющийся
платёжеспосо^бного
платёжеспосо^бностей
платёжеспосо^бность
платёжесп

In [25]:
test_words

['аакам',
 'ааками',
 'ааленец',
 'аама',
 'аамами',
 'аамов',
 'аамом',
 'аамы',
 'аангичами',
 'аангичах',
 'аангиче',
 'аангичи',
 'аангичу',
 'аарон',
 'аарона',
 'ааронов',
 'аароновцам',
 'аароновцах',
 'аароновцев',
 'аароновцем',
 'аароновщиной',
 'аароновщиною',
 'аароны',
 'аахен',
 'абаас',
 'абаасам',
 'абаасах',
 'абаасе',
 'абаасом',
 'абаданка',
 'абаданке',
 'абаданкой',
 'абаданцем',
 'абаддон',
 'абадзех',
 'абажур',
 'абажура',
 'абажурах',
 'абажурна',
 'абажурный',
 'абажуродержатель',
 'абажуры',
 'абазин',
 'абазинкам',
 'абазинками',
 'абазинку',
 'абазином',
 'абазинска',
 'абазински',
 'абазину',
 'абазинце',
 'абазию',
 'абазой',
 'абайя',
 'абаканец',
 'абаках',
 'абакостами',
 'абакостах',
 'абактериально',
 'абактериальный',
 'абактинальный',
 'абакумычами',
 'абакумыче',
 'абакумычем',
 'абалона',
 'абалонами',
 'абалоне',
 'абалонов',
 'абами',
 'абанамат',
 'абант',
 'абарг',
 'абаргам',
 'абаргу',
 'абарогнозия',
 'абарогнозом',
 'абат',
 'абатиса',
 '

In [23]:
list(map(lambda x: x.replace('^', ''), train_words))

['аак',
 'аака',
 'ааке',
 'ааки',
 'ааков',
 'ааком',
 'аам',
 'ааму',
 'аангича',
 'аангичам',
 'аароне',
 'аароновец',
 'аароновские',
 'аароновский',
 'аароновца',
 'аароновцами',
 'аароновце',
 'аароновцы',
 'аароновщин',
 'аароновщинами',
 'аароновщинах',
 'аароновщины',
 'аарону',
 'аахенец',
 'аахенский',
 'абаасами',
 'абаасов',
 'абаасу',
 'абаасы',
 'абадан',
 'абаданец',
 'абаданках',
 'абаданки',
 'абаданкою',
 'абаданку',
 'абаданские',
 'абаданский',
 'абаданца',
 'абаданцами',
 'абаданце',
 'абаданцев',
 'абаданцы',
 'абажурами',
 'абажурно',
 'абажурны',
 'абажуров',
 'абажуру',
 'абазиею',
 'абазина',
 'абазинам',
 'абазинки',
 'абазинкою',
 'абазинские',
 'абазинско',
 'абазинца',
 'абазинцу',
 'абазинцы',
 'абазия',
 'абазов',
 'абазом',
 'абаим',
 'абай',
 'абак',
 'абакам',
 'абаками',
 'абакан',
 'абакане',
 'абаканский',
 'абаковый',
 'абаком',
 'абакост',
 'абакостам',
 'абакосте',
 'абакостом',
 'абаксиальный',
 'абактэриальный',
 'абакум',
 'абакумыч',
 'абак

In [7]:
train_dataset = TrainDataset(train_words, tokenizer=get_item_list)
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True, collate_fn=train_collate_fn)

val_dataset = TrainDataset(val_words, tokenizer=get_item_list)
val_loader = DataLoader(val_dataset, batch_size=2048, shuffle=True, collate_fn=train_collate_fn)

In [14]:
model52 = Model52()
optimizer = AdamW(model52.parameters(), lr=1e-3)

single_model(
    model=model52,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    loss_function=nn.CrossEntropyLoss()
)

  0%|          | 0/230 [00:00<?, ?it/s]  3%|▎         | 6/230 [01:15<43:34, 11.67s/it]  

In [ ]:
test_words = words = read_words_from_file("public_test_stresses.txt")
test_dataset = InferenceDataset(test_words)
preds = []
test_loader = DataLoader(test_dataset, batch_size=4096, collate_fn=inference_collate_fn)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model52.to(device)
model52.eval()
total_train_loss = 0

preds = []
targets = []

dl_size = len(test_loader)


for batch in tqdm(test_loader):
    for key in batch:
        batch[key] = batch[key].to(device)

    with torch.no_grad():
        logits = model52(batch)
        preds.append(logits.argmax(dim=1))
preds = torch.cat(preds, dim=0)

In [ ]:
def insert_carot_after_vowel(w, ks):
    vowels = ["а", "е", "ё", "и", "о", "у", "ы", "э", "ю", "я"]
    words = []
    for k in ks:
        count = 0
        w1 = ""

        for char in w:
            w1 += char

            if char.lower() in vowels:
                count += 1
                if count == k:
                    w1 += "^"
        words.append(w1)

    for w1 in words:
        if "^" in w1:
            return w1

In [ ]:
sub = []
for idx, word in tqdm(enumerate(test_words)):
    stress_idx = preds[idx].item()
    sub.append(insert_carot_after_vowel(word, stress_idx))

In [ ]:
print(test_words[2])
print(sub[2])

In [ ]:
def write_to_file(sub, path):
    with open(path, "w") as file:
        for word in sub:
            file.write(word + "\n")

In [ ]:
write_to_file(sub, "/content/sample_submission.txt")